In [2]:
%display latex

In [3]:
N = 3
F = GF(2^N, 'x')
x = F.gen()

In [4]:
basis = [x^3, x^5, x^6]

In [8]:
def toInt(k):
    return list(F).index(k)

In [9]:
def graph(pts):
    m = zero_matrix(SR, 2^N, 2^N)
    for p in pts:
        m[toInt(p[0]), toInt(p[1])] = 1
    return m

The exceptional curve in question is given by 
$$
\gamma = \{
(0, 0), (\sigma^4, 0), (\sigma^4, \sigma^5), 
(\sigma^3, \sigma^7), (\sigma^3, \sigma^4), (\sigma^6, \sigma^4), 
(\sigma^6, \sigma^7), (0, \sigma^5)
\}.
$$

In [10]:
curve = [
    (F(0),F(0)), (x^4,F(0)), (x^4,x^5), (x^3,x^7),
    (x^3,x^4), (x^6,x^4), (x^6,x^7), (F(0),x^5)
]

In [11]:
len(curve)

8

In [13]:
graph(curve)

[1 0 0 0 0 1 0 0]
[0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 1]
[1 0 0 0 0 1 0 0]
[0 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 1]
[0 0 0 0 0 0 0 0]

We know that this curve is closed under addition and commutative, i.e.,
$$
Tr(\alpha \beta') = Tr(\alpha' \beta),
\quad
\forall (\alpha,\beta), (\alpha',\beta') \in \gamma.
$$

In [12]:
# closed under addition
for p1 in curve:
    for p2 in curve:
        p3 = (p1[0] + p2[0], p1[1] + p2[1])
        if p3 not in curve:
            raise Exception

In [16]:
# commutativity
for p1 in curve:
    for p2 in curve:
        if (p1[0] * p2[1]).trace() != (p1[1] * p2[0]).trace():
            raise Exception(p1, p2)

The following method involves finding all the linear transformations from the field $F$ to the curve that preserve the additivity condition. This is brute force, although later we define it in a simple manner:

In [17]:
def components(k):
    return vector([(k * b).trace() for b in basis])

In [18]:
def assemble(v):
    s = 0
    for i, b in enumerate(basis):
        s += v[i] * b
    return s

In [19]:
for k in F:
    if assemble(components(k)) != k:
        raise Exception

In [20]:
M_alpha = []
good_alphas = []
for M in Matrix(Integers(2), 3, 3).parent():
    alphas = []
    for tau in F:
        v = components(tau)
        alphas.append(assemble(M * v))
    if set(alphas) == set([p[0] for p in curve]):
        good_alphas.append(alphas)
        M_alpha.append(M)

M_beta = []
good_betas = []
for M in Matrix(Integers(2), 3, 3).parent():
    betas = []
    for tau in F:
        v = components(tau)
        betas.append(assemble(M * v))
    if set(betas) == set([p[1] for p in curve]):
        good_betas.append(betas)
        M_beta.append(M)

print(len(good_alphas), len(good_betas))

42 42


In [21]:
Ms = []
params = []
for i, alphas in enumerate(good_alphas):
    for j, betas in enumerate(good_betas):
        param = list(zip(alphas, betas))
        if set(param) == set(curve):
            params.append(param)
            Ms.append((M_alpha[i], M_beta[j]))
len(params)

168

In [22]:
params[0]

[(0, 0),
 (x^2 + 1, 1),
 (x^2 + x, 0),
 (x + 1, x^2 + x),
 (x + 1, 1),
 (x^2 + 1, x^2 + x),
 (0, x^2 + x + 1),
 (x^2 + x, x^2 + x + 1)]

In [23]:
curve

[(0, 0),
 (x^2 + x, 0),
 (x^2 + x, x^2 + x + 1),
 (x + 1, 1),
 (x + 1, x^2 + x),
 (x^2 + 1, x^2 + x),
 (x^2 + 1, 1),
 (0, x^2 + x + 1)]

It works, but there is a better way...

We can build any of the 168 parametrizations by selecting three generating elements of the curve, $p_1,p_2,p_3 \in \gamma$ and using the following expression:
$$
(\alpha(\tau), \beta(\tau))
= Tr(\tau x^3) p_1 + Tr(\tau x^5) p_2 + Tr(\tau x^6) p_3.
$$

To find the generating elements of the curve we can select two distinct points and then select a third one that is distinct from the first two and is also not equal to their sum.

In [33]:
curve

[(0, 0),
 (x^2 + x, 0),
 (x^2 + x, x^2 + x + 1),
 (x + 1, 1),
 (x + 1, x^2 + x),
 (x^2 + 1, x^2 + x),
 (x^2 + 1, 1),
 (0, x^2 + x + 1)]

For example we choose the following points:
$$
p_1 = (x^2+x,0), \quad
p_2 = (x+1,1), \quad
p_3 = (x^2+x,x^2+x+1).
$$

In [34]:
def gamma(tau, gens):
    comps = components(tau)
    p = vector([F(0), F(0)])
    for i, g in enumerate(gens):
        p += comps[i] * g
    return tuple(p)

In [35]:
gens = [vector([x^2+x,F(0)]), vector([x+1,F(1)]), vector([x^2+x,x^2+x+1])]

In [37]:
[gamma(tau, gens) for tau in F]

[(0, 0),
 (x^2 + 1, x^2 + x),
 (x^2 + 1, 1),
 (x^2 + x, 0),
 (0, x^2 + x + 1),
 (x + 1, 1),
 (x^2 + x, x^2 + x + 1),
 (x + 1, x^2 + x)]

In [39]:
def powers(pts):
    return [(toInt(p[0]), toInt(p[1])) for p in pts]

In [40]:
powers([gamma(tau, gens) for tau in F])

[(0, 0), (6, 4), (6, 7), (4, 0), (0, 5), (3, 7), (4, 5), (3, 4)]

In [42]:
set(curve) == set([gamma(tau, gens) for tau in F])

True

So our example of parametrization works fine. The following is code that Jorge created to obtain all the possible generators and from there test that each parametrization is correct. 

In [51]:
def listminus(rs, notrs):
    res = []
    for s in rs:
        if s not in notrs:
            res.append(s)
    return(res)

generators = []
set1 = listminus(curve,[(F(0),F(0))])
for p1 in set1:
    set2 = listminus(set1,[p1])
    for p2 in set2:
        set3 = listminus(set2, [p2, (p1[0]+p2[0],p1[1]+p2[1])])
        for p3 in set3:
            generators.append([vector(p1), vector(p2), vector(p3)])

In [52]:
len(generators)

168

In [53]:
for c in [[gamma(tau, gen) for tau in F] for gen in generators]:
    if set(c) != set(curve):
        raise Exception

The paramtrization idea works perfectly. The following is confirmation that we can form the Wigner kernel using the displacement expression now that we have have a paramtrization of the degenerate curve. Using this expression we can obtain all eigenvectors and I later test that it does create an eigenbasis of the displacement operators indexed by the elements of the curve.

In [54]:
chi = lambda k: exp(pi * I * int(k.trace()))

y = var('y')

# compute the coefficients for the basis elements,
def solve_basis(basis, curve):
    sols = []
    for k in basis:
        sol = solve(chi(k * curve(k)) == y^2, y, solution_dict=True)
        sols.append(sol[1]) # positive solutions only (choice)
    return sols

# after computing the solutions we can simply
# get them for each basis element.
def basis_c(a, l, sols):
    if a == 0:
        return 1
    return sols[l][y]

# Compute an arbitrary coefficient c_{\alpha,f} for a 
# given curve f using the general formula.
def c(alpha, curve, sols=None):
    if not sols:
        sols = solve_basis(basis, curve)
    
    # Expand alpha in the basis
    comps = components(alpha)

    # Apply the formula
    s1 = 0
    for k in range(N-1):
        s2 = 0
        for j in range(k+1, N):
            s2 += comps[j] * basis[j]
        s1 += s2 * curve(comps[k] * basis[k])
        
    return chi(s1) * prod([basis_c(a, l, sols) for l, a in enumerate(comps)])

def sign_perm(sols, perm=None):
    if perm:
        for k, sol in enumerate(sols):
            sols[k][y] = perm[k] * sols[k][y]
    return sols

def ray(mu):
    return lambda t: mu * t

def PS(curve, perms):
    phase_space = zero_matrix(SR, 2^N, 2^N)
    phase_space[0,:] = 1 # vertical line
    for j, mu in enumerate(F): # iterate through the curve parametr
        # same sign choice for a fixed curve parameter
        sols = sign_perm(solve_basis(basis, curve(mu)), perms[j])
        for i, a in enumerate(F):
            # loop through alpha to obtain coefficient and
            # corresponding point
            coeff = c(a, curve(mu), sols)
            b = curve(mu)(a)
            phase_space[i, toInt(b)] = coeff
    return phase_space

In [55]:
ps = PS(ray, [[1,1,1]] * 8)
ps

[ 1  1  1  1  1  1  1  1]
[ 1 -1 -I  1 -I  I  I -1]
[ 1 -I -1  I -I  I  1 -1]
[ 1  1  I  I  I  1  1  I]
[ 1 -I -I  I -1  1  I -1]
[ 1  I  I  1  1  I  1  I]
[ 1  I  1  1  I  1  I  I]
[ 1 -1 -1  I -1  I  I -I]

In [56]:
def Proj(u, v=None):
    if not v:
        v = u
    return u.tensor_product(v.conjugate_transpose())

Id = identity_matrix(2^N)

def Fourier():
    s = zero_matrix(SR, 2^N, 2^N)
    for i, a in enumerate(F):
        for j, b in enumerate(F):
            s[i,j] = chi(a * b) / sqrt(2^N)
    return s
FF = Fourier()

def phi(a, b):
    return ps[toInt(a), toInt(b)]

def Z(a):
    return diagonal_matrix([chi(a * k) for k in F])

def X(b):
    return FF.conjugate_transpose() * Z(b) * FF

def D(a, b):
    return phi(a, b) * Z(a) * X(b)

In [57]:
# Testing that a parametrization gives us 
# the abelian property. Of course this works
# because we already know that the curve is abelian!
k = 0
for t1 in F:
    for t2 in F:
        p1 = gamma(t1, generators[k])
        p2 = gamma(t2, generators[k])
        d1 = D(*p1)
        d2 = D(*p2)
        d3 = D(*(vector(p1) + vector(p2)))
        if d1 * d2 != d3:
            raise Exception

The following expression produces the projection operator corresponding to the $\kappa$-th eigenvector of the eigenbasis of the displacements operatos corresponding to $\gamma$. 

In [58]:
def eig_proj(k, j):
    eig = zero_matrix(SR, 2^N, 2^N)
    for tau in F:
        eig += chi(tau * k) * D(*gamma(tau, generators[j])) / 2^N
    return eig

In [59]:
eig = eig_proj(F(0), 123)

In [60]:
# testing that it is an eigenvector
dop = D(*gamma(x, generators[0]))
dop * eig * dop.conjugate_transpose() == eig

True

In [61]:
# testing that they all are eigenvectors
# of the D's.
for tau in F:
    for k in F:
        eig = eig_proj(k, 123)
        dop = D(*gamma(tau, generators[0]))
        if dop * eig * dop.conjugate_transpose() != eig:
            raise Exception

In [62]:
for k in F:
    print(eig_proj(k, 123), '\n')

[   1/4      0      0      0  1/4*I    1/4      0  1/4*I]
[     0      0      0      0      0      0      0      0]
[     0      0      0      0      0      0      0      0]
[     0      0      0      0      0      0      0      0]
[-1/4*I      0      0      0    1/4 -1/4*I      0    1/4]
[   1/4      0      0      0  1/4*I    1/4      0  1/4*I]
[     0      0      0      0      0      0      0      0]
[-1/4*I      0      0      0    1/4 -1/4*I      0    1/4] 

[     0      0      0      0      0      0      0      0]
[     0    1/4  1/4*I -1/4*I      0      0   -1/4      0]
[     0 -1/4*I    1/4   -1/4      0      0  1/4*I      0]
[     0  1/4*I   -1/4    1/4      0      0 -1/4*I      0]
[     0      0      0      0      0      0      0      0]
[     0      0      0      0      0      0      0      0]
[     0   -1/4 -1/4*I  1/4*I      0      0    1/4      0]
[     0      0      0      0      0      0      0      0] 

[     0      0      0      0      0      0      0      0]
[     0   

In [63]:
V = CC^64
V

Vector space of dimension 64 over Complex Field with 53 bits of precision

In [64]:
V.linear_dependence([vector(eig_proj(k, 123).list()) for k in F])

[

]

The set of eigenvectors is an eigenbasis. The Wigner function should give us delta functions since the displacement operators have matching phases with the rays.

In [77]:
def Wigner(a, b):
    op = zero_matrix(SR, 2^N, 2^N)
    for ga in F:
        for de in F:
            op += chi(ga * b + de * a) * D(ga, de)
    return op / 2^N

In [84]:
def WignerMatrix(state, kernels=None):
    m = zero_matrix(SR, 2^N, 2^N)
    for a in F:
        for b in F:
            if kernels:
                kernel = kernels[toInt(a)][toInt(b)]
            else:
                kernel = Wigner(a, b)
            m[toInt(a), toInt(b)] = (kernel * state).trace() / 2^N
    return m

In [97]:
kernels = []
for a in F: 
    row = [Wigner(a, b) for b in F]
    kernels.append(row)

In [101]:
WignerMatrix(eig_proj(0, 0), kernels)

[1/8   0   0   0   0 1/8   0   0]
[  0   0   0   0   0   0   0   0]
[  0   0   0   0   0   0   0   0]
[  0   0   0   0 1/8   0   0 1/8]
[1/8   0   0   0   0 1/8   0   0]
[  0   0   0   0   0   0   0   0]
[  0   0   0   0 1/8   0   0 1/8]
[  0   0   0   0   0   0   0   0]

In [106]:
for k in F:
    print(WignerMatrix(eig_proj(k, 0), kernels), '\n')

[1/8   0   0   0   0 1/8   0   0]
[  0   0   0   0   0   0   0   0]
[  0   0   0   0   0   0   0   0]
[  0   0   0   0 1/8   0   0 1/8]
[1/8   0   0   0   0 1/8   0   0]
[  0   0   0   0   0   0   0   0]
[  0   0   0   0 1/8   0   0 1/8]
[  0   0   0   0   0   0   0   0] 

[  0   0   0   0   0   0   0   0]
[  0   0   0   0 1/8   0   0 1/8]
[  0   0   0   0 1/8   0   0 1/8]
[  0   0   0   0   0   0   0   0]
[  0   0   0   0   0   0   0   0]
[1/8   0   0   0   0 1/8   0   0]
[  0   0   0   0   0   0   0   0]
[1/8   0   0   0   0 1/8   0   0] 

[  0 1/8   0   0   0   0 1/8   0]
[  0   0   0   0   0   0   0   0]
[  0   0   0   0   0   0   0   0]
[  0   0 1/8 1/8   0   0   0   0]
[  0 1/8   0   0   0   0 1/8   0]
[  0   0   0   0   0   0   0   0]
[  0   0 1/8 1/8   0   0   0   0]
[  0   0   0   0   0   0   0   0] 

[  0   0   0   0   0   0   0   0]
[  0 1/8   0   0   0   0 1/8   0]
[  0 1/8   0   0   0   0 1/8   0]
[  0   0   0   0   0   0   0   0]
[  0   0   0   0   0   0   0   0]
[  0   0

We obtain delta functions as we wanted!

Now, Sainz found a parametrization in a simpler expression, where she basically fit the parameters of a general curve to the points of the degenerate curve in question:
$$
\alpha(\tau) = \sigma \tau^2 + \sigma^4 \tau^4,
\quad
\beta(\tau) = \sigma \tau + \sigma^2 \tau^4.
$$
First we test that this curve is an actual parametrization.

In [113]:
sainz = [(x*t^2 + x^4*t^4, x*t + x^2*t^4) for t in F]
sainz

[(0, 0),
 (1, 1),
 (0, 0),
 (x^2 + 1, x^2 + x + 1),
 (1, 1),
 (x^2 + 1, x^2 + x + 1),
 (x^2, x^2 + x),
 (x^2, x^2 + x)]

In [109]:
set(sainz) == set(curve)

False

In [114]:
ern = [gamma(tau, generators[0]) for tau in F]
ern

[(0, 0),
 (x^2 + 1, x^2 + x),
 (0, x^2 + x + 1),
 (x^2 + x, 0),
 (x^2 + 1, 1),
 (x^2 + x, x^2 + x + 1),
 (x + 1, 1),
 (x + 1, x^2 + x)]

In [115]:
set(ern) == set(curve)

True

So first of all her parametrization does not work. So now let's find a better expression for our parametrization. For starters, given a set of generators of the curve, $x = (x_1,x_2), y = (y_1,y_2)$ and $z = (z_1,z_2)$, we can obtain a parametrization of and additive curve in the following way:
$$
\alpha(\tau) = Tr(\tau \sigma^3) x_1 + Tr(\tau \sigma^5) y_1 + Tr(\tau \sigma^6) z_1,
\quad
\beta(\tau) = Tr(\tau \sigma^3) x_2 + Tr(\tau \sigma^5) y_2 + Tr(\tau \sigma^6) z_2.
$$

In [132]:
# let's choose this generator to comply with the pdf I sent Klimov
g = [vector([x^4,F(0)]), vector([x^3,x^7]), vector([x^4,x^5])]
g

[(x^2 + x, 0), (x + 1, 1), (x^2 + x, x^2 + x + 1)]

In [133]:
set([gamma(tau, g) for tau in F]) == set(curve)

True

In [136]:
powers([gamma(tau, g) for tau in F])

[(0, 0), (6, 4), (6, 7), (4, 0), (0, 5), (3, 7), (4, 5), (3, 4)]

Now I'm going to try to use SageMath to simplify the expresion for the $\alpha$ and $\beta$ components of this particular paramtrization:

In [141]:
g[0][0] * x^3 + g[1][0] * x^5 + g[2][0] * x^6

0

In [142]:
g[0][0] * x^6 + g[1][0] * x^3 + g[2][0] * x^5

x

In [143]:
g[0][0] * x^5 + g[1][0] * x^6 + g[2][0] * x^3

1

So the expression for the first component is given by $\alpha(\tau) = \sigma \tau^2 + \tau^4.$
Let's verify:

In [149]:
for tau in F:
    if gamma(tau, g)[0] != x * tau^2 + tau^4:
        raise Exception(tau)

In [153]:
g[0][1] * x^3 + g[1][1] * x^5 + g[2][1] * x^6

1

In [154]:
g[0][1] * x^6 + g[1][1] * x^3 + g[2][1] * x^5

0

In [155]:
g[0][1] * x^5 + g[1][1] * x^6 + g[2][1] * x^3

x^2 + x + 1

The expression for the second component is $\beta(\tau) = \tau + \sigma^5 \tau^4$. Let's verify:

In [158]:
for tau in F:
    if gamma(tau, g)[1] != tau + x^5 * tau^4:
        raise Exception(tau)

So an actually valid parametrization is given by:
$$
\alpha(\tau) = \sigma \tau^2 + \tau^4,
\quad
\beta(\tau) = \tau + \sigma^5 \tau^4.
$$

In [159]:
ern_param = lambda t: (x * t^2 + t^4, t + x^5 * t^4)

In [161]:
set([ern_param(tau) for tau in F]) == set(curve)

True

In [167]:
# additivity of the curve
for t1 in F:
    for t2 in F:
        if ern_param(t1)[0] + ern_param(t2)[0] != ern_param(t1+t2)[0]:
            raise Exception
        if ern_param(t1)[1] + ern_param(t2)[1] != ern_param(t1+t2)[1]:
            raise Exception

In [169]:
# commutativity condition
for t1 in F:
    p1 = ern_param(t1)
    p2 = ern_param(t2)
    for t2 in F:
        if (p1[0] * p2[1]).trace() != (p1[1] * p2[0]).trace():
            raise Exception

Now I am interested in the following property:
$$
\chi(\alpha(\tau) \kappa) = \chi(\alpha(\kappa) \tau),
$$
for all $\tau \in F$ and an *arbitrary* $\kappa$, we ask the same for the second component.

In [170]:
for tau in F:
    for beta in F:
        if (gamma(tau, g)[0] * beta).trace() != (gamma(beta, g)[0] * tau).trace():
            raise Exception(tau, beta)

Exception: (x, x^2)

In [173]:
gamma(x, g)[0] * x^2

x

In [174]:
gamma(x^2, g)[0] * x

1

So as Klimov states, the property does not hold for the *whole* field. 

Let's recap, we have a valid parametrization of our doubly degerenate curve:
$$
\alpha(\tau) = \sigma \tau^2 + \tau^4,
\quad
\beta(\tau) = \tau + \sigma^5 \tau^4.
$$

This curve can be expressed analytically by the following equation:
$$
\sigma^5 \beta + \beta^2 = \sigma^6 \alpha + \sigma^2 \alpha^2,
$$
for $\alpha,\beta$ that satisfy the structural equations:
$$
Tr(\sigma^4\beta) = 0,
\quad
Tr(\sigma^5 \alpha) = 0.
$$

In [183]:
# verify that the parametrization satisfies
# the equation:
for tau in F:
    p = gamma(tau, g)
    alpha, beta = p
    if x^5 * beta + beta^2 != x^6 * alpha + x^2 * alpha^2:
        raise Exception

In [184]:
# verify that the parametrization satisfies
# the structural equations:
for tau in F:
    p = gamma(tau, g)
    alpha, beta = p
    if (x^4 * beta).trace() != 0:
        raise Exception
    if (x^5 * alpha).trace() != 0:
        raise Exception

So the trace property does not hold for all values of the field. The parametrization obviosly satisfies the curve equation and the structural equations, the later give us an admisible set of points, so, what is the relation between the trace property and the admisible set?